# Model fine-tuning (Llama-2-7b-chat)
for predicting esg score

Reference: https://www.datacamp.com/tutorial/fine-tuning-llama-2

In [7]:
# pip install tensorboard

In [ ]:
# pip install datasets

In [3]:
# pip install trl

In [1]:
import os
import re
import torch
import pandas as pd
import warnings
from datasets import Dataset
from datetime import datetime
from peft import LoraConfig
from tensorboard import notebook
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

os.chdir("../../") # current location is /notebooks/llama

ModuleNotFoundError: No module named 'sklearn'

In [2]:
import pandas as pd
import os
os.chdir("../../") # current location is /notebooks/llama

### Load and preprocess training data

In [3]:
train_path = "data/processed/train_esg_shortened.csv"
df_train = pd.read_csv(train_path)
print(df_train.shape)

(547, 6)


In [4]:
pwd

'c:\\Users\\HERO\\OneDrive - connect.hku.hk\\!HKU Course\\FINA4350 Text Analytics and Natural Language Processing in Finance and Fintech\\proj1\\fina-nlp-project\\notebooks'

In [4]:
df_train.head()

,symbol,year,quarter,transcript_esg,esg_score,esg_risk_level
0,A,2022,1,thank emily welcome everyone agilents conferen...,15.0,Low
1,A,2022,3,thank hannah welcome everyone agilents confere...,15.0,Low
2,AAPL,2022,1,good day welcome apple q fy earnings conferenc...,17.0,Low
3,AAPL,2022,2,good day welcome apple q fy earnings conferenc...,17.0,Low
4,AAPL,2022,3,good day welcome apple q fy earnings conferenc...,17.0,Low


In [5]:
# validate the range of score
esg_min = df_train['esg_score'].min()
esg_max = df_train['esg_score'].max()

print(f"Minimum ESG risk level: {esg_min}")
print(f"Maximum ESG risk level: {esg_max}")

Minimum ESG risk level: 9.0
Maximum ESG risk level: 42.0


In [6]:
def combine_question_answer(transcript_esg, esg_score):
    system_msg = "<<SYS>>\n" \
    + "Your task is to predict the ESG score based on earning calls transcripts. The ESG score should be a number between 0 and 50, and should be precise to 1 decimal place.\n" \
    + "The ESG Risk Ratings are categorized across five risk levels: negligible (0-10), low (10-20), medium (20-30), high (30-40), and severe (40+).\n" \
    + "When provided with an earning calls transcript, you should output the predicted ESG score.\n" \
    + "Please format your answer as a single number with 1 decimal place, such as \'12.3'.\n" \
    + " (40+ means severe, if you think the score should go over 50, go on. but just think twice. You don't need to give explanation.).\n" \
    + " Remember, only tell me the number in your final answer. Thank you.\n" \
    + "<</SYS>>\n\n"
    prompt = f"<s>[INST] {system_msg}###User: {transcript_esg}###You: [/INST] {esg_score}</s>"
    return prompt

df_train["text"] = df_train.apply(
    lambda row: combine_question_answer(row.transcript_esg, row.esg_score), axis=1
)
df_train[["transcript_esg", "esg_score", "text"]].head()

,transcript_esg,esg_score,esg_risk_level,text
0,thank emily welcome everyone agilents conferen...,15.0,Low,<s>[INST] <<SYS>>\nYour task is to predict the...
1,thank hannah welcome everyone agilents confere...,15.0,Low,<s>[INST] <<SYS>>\nYour task is to predict the...
2,good day welcome apple q fy earnings conferenc...,17.0,Low,<s>[INST] <<SYS>>\nYour task is to predict the...
3,good day welcome apple q fy earnings conferenc...,17.0,Low,<s>[INST] <<SYS>>\nYour task is to predict the...
4,good day welcome apple q fy earnings conferenc...,17.0,Low,<s>[INST] <<SYS>>\nYour task is to predict the...


In [7]:
df_train = df_train.sample(frac=1)
dataset_train = Dataset.from_pandas(df_train[["text"]][:int(df_train.shape[0]*0.8)])
dataset_eval = Dataset.from_pandas(df_train[["text"]][int(df_train.shape[0]*0.8):])
for text in dataset_train:
    print(text)
    break

{'text': '<s>[INST] <<SYS>>\nYour task is to predict the ESG risk level based on earning calls transcripts.\nThe ESG Risk Ratings are categorized across five risk levels: negligible (0-10), low (10-20), medium (20-30), high (30-40), and severe (40+).\nThe predicted risk level should always be one of the following: [\'Negligible\', \'Low\', \'Medium\', \'High\', \'Severe\'].\nWhen provided with an earning calls transcript, you should output the predicted ESG risk level along with the corresponding ESG score.\nPlease format your answer as "{score}, {label}", where the score is the ESG score rounded to one decimal place, and the label is the predicted ESG risk level.\nEnsure that your predicted risk level is consistent with the predicted score.\n<</SYS>>\n\n###User: good day welcome apple q fy earnings conference call today call recorded time opening remark introduction would like turn call tejas gala director investor relation corporate finance please go ahead set alltime record develope

In [9]:
dataset_eval

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 110
})

### Load tokenizer and base model

In [14]:
base_model = "meta-llama/Llama-2-7b-chat-hf"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-66292522-65f3051b1c7633b73bc31b4e;16883d14-67d5-4191-ad99-098c548c1fab)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must be authenticated to access it.

In [8]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

### Set params for PEFT fine-tuning

In [11]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
timestamp = datetime.now().strftime("%y%m%d_%H%M")
training_params = TrainingArguments(
    output_dir=f"./results/results_{timestamp}",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    evaluation_strategy="steps",
    save_steps=10,
    save_total_limit=5,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    load_best_model_at_end=True,
)

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/356 [00:00<?, ? examples/s]

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

### Model fine-tuning

In [14]:
trainer.train()

Step,Training Loss,Validation Loss
10,2.556700,2.469196
20,2.349600,2.118536
30,2.038300,1.894948
40,1.765200,1.851612
50,1.720600,1.831419
60,1.827500,1.822108
70,1.733800,1.816305
80,1.810200,1.809533
90,1.573700,1.808826
100,1.813000,1.804062


TrainOutput(global_step=267, training_loss=1.7556427837757582, metrics={'train_runtime': 3612.157, 'train_samples_per_second': 0.296, 'train_steps_per_second': 0.074, 'total_flos': 1.6067174682476544e+16, 'train_loss': 1.7556427837757582, 'epoch': 3.0})

In [15]:
# edited, added "llama_" at the beginning

trainer.model.save_pretrained(f"llama_models/model_{timestamp}")
trainer.tokenizer.save_pretrained(f"llama_models/model_{timestamp}")

('models/model_240411_0952/tokenizer_config.json',
 'models/model_240411_0952/special_tokens_map.json',
 'models/model_240411_0952/tokenizer.model',
 'models/model_240411_0952/added_tokens.json',
 'models/model_240411_0952/tokenizer.json')

### Model Inferencing & Evaluation

In [ ]:
test_path = "data/processed/test_esg_shortened.csv"
df_test = pd.read_csv(test_path)
print(df_test.shape)

In [ ]:
df_test.head()

In [ ]:
def get_question_prompt(transcript_esg):
    system_msg = "<<SYS>>\n" \
        + "Your task is to predict the ESG score based on earning calls transcripts. The ESG score should be a number between 0 and 50, and should be precise to 1 decimal place.\n" \
        + "The ESG Risk Ratings are categorized across five risk levels: negligible (0-10), low (10-20), medium (20-30), high (30-40), and severe (40+).\n" \
        + "When provided with an earning calls transcript, you should output the predicted ESG score.\n" \
        + "Please format your answer as a single number with 1 decimal place, such as \'12.3'.\n" \
        + " (40+ means severe, if you think the score should go over 50, go on. but just think twice. You don't need to give explanation.).\n" \
        + " Remember, only tell me the number in your final answer. Thank you.\n" \
        + "<</SYS>>\n\n"
    prompt = f"<s>[INST] {system_msg}###User: {transcript_esg}###You: [/INST]</s>"
    return prompt

In [ ]:
df_train["prompt"] = df_train["transcript_esg"].apply(get_question_prompt)
df_train[["transcript_esg", "prompt"]].head()

In [ ]:
df_test["prompt"] = df_test["transcript_esg"].apply(get_question_prompt)
df_test[["transcript_esg", "prompt"]].head()

In [ ]:
pipe = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    device_map="auto"
)

def evaluate_and_save_results(model, df, filename):
    # model.eval()
    # loader = DataLoader(dataset, batch_size=16)
    # all_predictions = []
    # with torch.no_grad():
    #     for batch in loader:
    #         input_ids = batch['input_ids'].to(device)
    #         attention_mask = batch['attention_mask'].to(device)
    #         outputs = model(input_ids, attention_mask)
    #         predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
    #         all_predictions.extend(predictions)

    # # Convert numerical predictions back to label names using the inverse transformation of the label encoder
    # predicted_labels = label_encoder.inverse_transform(all_predictions)

    # get generated text results as the predicted esg score
    # input to llama2 to generate text
    chunk_size = 4
    num_chunks = df.shape[0] // chunk_size + int(df.shape[0] % chunk_size > 0)
    results = []
    with warnings.catch_warnings(category=UserWarning):
        warnings.filterwarnings("ignore")
        for i in tqdm(range(num_chunks)):
            result = pipe(df["prompt"].to_list()[i*chunk_size: (i+1)*chunk_size])
            results.extend(result)
    # print(results[0])

    # extract the answers from generated text
    df["generatedText"] = [res[0]["generated_text"] for res in results]
    df["predicted_esg_score"] = df["generatedText"].apply(
        lambda text: re.split(r"\[\/INST\]\s*", text, maxsplit=1)[1]
    )
    df.head()

    # Calculate MSE for the score prediction
    mse = mean_squared_error(df['esg_score'], df['predicted_esg_score'])
    print(f"Test MSE: {mse}")

    # Save to CSV
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")
    return mse

In [ ]:
train_accuracy = evaluate_and_save_results(model, df_train.copy(), f"data/results/train_score_results_{timestamp}.csv")
test_accuracy = evaluate_and_save_results(model, df_test.copy(), f"data/results/test_score_results_{timestamp}.csv")

### Model inferencing demo
(from APAI... should go to ...test.ipynb instead of here)

In [16]:
# def get_question_prompt(question):
#     system_msg = "<<SYS>>\n" \
#         + "You are a helpful and honest psychologist." \
#         + "Respond to the following question from the user." \
#         + "Your answers should not include any critical, dangerous or illegal content." \
#         + "Your answers should be unbiased and encouraging.\n" \
#         + "<</SYS>>\n\n"
#     prompt = f"<s>[INST] {system_msg}###User: {question}###You: [/INST]"
#     return prompt

# question = "I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?"
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
# result = pipe(get_question_prompt(question))
# print(result[0]['generated_text'])

<s>[INST] <<SYS>>
You are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.
<</SYS>>

###User: I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?###You: [/INST]I think you are on the right track.   You are worried about your school work, and that is a good thing.   You are aware of the potential consequences of not completing your work on time, and that is also a good thing.   I think you should talk to your teacher about your concerns.   They may be able to help you come up with a plan to help you get your work done on time.   Also, you can also try breaking down your work into smaller tasks, and then set deadlines for yourself to complete each one.   You can also try to find a quiet place to study, and try to avoid distractions.   I h

### Visualization

In [4]:
# timestamp = "240411_0952"
# log_dir = f"results/results_{timestamp}/runs"
# notebook.start(f"--logdir {log_dir} --port 4000")